In [8]:
#r @"..\CHSearch\bin\Debug\Newtonsoft.Json.dll"
#r @"..\CHSearch\bin\Debug\CHSearch.exe"
#r @"..\CHSearch\bin\Debug\PocoGeneration.exe"
#r @"..\CHSearch\bin\Debug\LiteDB.dll"
#r @"C:\Program Files (x86)\Reference Assemblies\Microsoft\Framework\.NETFramework\v4.6.1\System.Net.Http.dll"

In [9]:
open System
open Onnea
open Newtonsoft.Json
open Onnea.Domain
open System.IO

In [10]:
System.IO.Directory.SetCurrentDirectory(@"C:\Projects\CHSearch\CHSearch")

In [11]:
let db = Commands.GetDatabase( @"C:\temp\CHSearch\db\temp.db" )

In [12]:
let adf = Commands.Fetch( db, from = 314330, count = 15 );
let fetched = adf |> Seq.toList //|> Seq.length
let fromWeb = fetched |> Seq.where (fun r -> r.WasFetchedFromWeb) |> Seq.toList

Fetching company info from 314330 to 314344
...............
Waiting for the fetcher to finish...
Done fetching company info from 314330 to 314344


In [16]:
let allCompanies = Commands.GetCompanies( db, fun _ -> true ) |> Seq.filter (fun c -> not c.DoesNotExist) |> List.ofSeq
allCompanies |> Seq.map (fun c -> c.CompanyName)  |> Seq.take 10

seq
  ["POST & TIMES SERIES LIMITED"; "SHS INTERNATIONAL LTD";
   "NEWSQUEST (LANCS FREE) LIMITED"; "PERSHORE CLUB;LIMITED(THE)"; ...]

In [17]:
let companiesB = Commands.GetCompaniesWhere( db, "CompanyName",
                  fun c -> c.AsString <> null && c.AsString.StartsWith( "B" ) )
let companiesC = Commands.GetCompaniesWhere( db, "_id", // aka "CompanyInfoId"
                  fun c -> c.AsInt32 = 314128 ) |> List.ofSeq
let activeB = companiesB 
              //|> Seq.filter (fun c -> c.CompanyStatus = "active") 
              |> Seq.toList

In [18]:
let fh = 
  allCompanies 
  |> Seq.map (fun c -> c.CompanyName, Commands.GetFilingHistory( db, c, DateTime.Parse( "2018-05-22" )))
fh |> Seq.length

74

In [20]:
let fhsWithFullAccounts 
  = fh |> Seq.where (fun h -> (h |> snd).Items 
                             |> Seq.exists (fun i -> i.Description.ToLower().Contains("full")))
       |> List.ofSeq

fhsWithFullAccounts 
   |> Seq.last
   |> fun fhi -> (fhi |> snd).Items |> Seq.filter (fun i -> i.Links <> null) |> Seq.last 
   |> fun i -> Commands.GetDocument(i)
   |> fun doco -> printfn "%s" doco.TextFile

C:\temp\CHSearch\documents\text\https___document-api.companieshouse.gov.uk_document_2xEUoeA1ewueRjAydrwdVOnyXuP-EEqnmSj-3mwJNwE.txt


In [22]:
let docos = 
    fhsWithFullAccounts 
    |> Seq.map (fun fhi -> (fhi |> fst), 
                           (fhi |> snd).Items |> Seq.map (fun i -> Commands.GetDocument(i) ) )
    |> List.ofSeq

let lines = docos.Item 5 |> snd |> Seq.map( fun fhd -> fhd.TextFile |> File.ReadAllLines )

lines |> Seq.map (Seq.head) |> Seq.take 8 |> List.ofSeq

["DISSOLVED"; "Inaorf‘rdanoewilh BLUEPRINT"; "lnaooorda oewith"; "Section 94";
 "In accordance with"; "In accordance With"; "Section 89(3) Form 4.70";
 "CHFP028"]